# Predicting Point Spread in Men's NCAA Basketball

We will use linear regression to predict the point spread of men's NCAA basketball games based on a number of relavent features from past games.

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Read Compact Data

In [2]:
# Load regular season compact results
reg_season_df = pd.read_csv('./data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
reg_season_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
161547,2019,132,1153,69,1222,57,N,0
161548,2019,132,1209,73,1426,64,N,0
161549,2019,132,1277,65,1276,60,N,0
161550,2019,132,1387,55,1382,53,N,0


## Create Relevant Features

In [3]:
## Create Features

# Find number of wins for each team in each season
numWins = reg_season_df.groupby(['Season', 'WTeamID'])['DayNum'].nunique()
df_win = pd.DataFrame(index=numWins.index, data=numWins)
df_win.reset_index(inplace=True)
df_win = df_win.rename(columns={'WTeamID': 'TeamID', 'DayNum': 'NumWins'})

# Find number of losses for each team
numLosses = reg_season_df.groupby(['Season', 'LTeamID'])['DayNum'].nunique()
df_loss = pd.DataFrame(index=numLosses.index, data=numLosses)
df_loss.reset_index(inplace=True)
df_loss = df_loss.rename(columns={'LTeamID': 'TeamID', 'DayNum': 'NumLosses'})

train_df = pd.merge(df_win, df_loss, how='inner', on=['TeamID', 'Season'])

# Find win percentage for each team
train_df['WinPercentage'] = train_df['NumWins'] / (train_df['NumWins'] + train_df['NumLosses'])

# Find average win score
avgWinScore = reg_season_df.groupby(['Season', 'WTeamID'])['WScore'].mean()
df_win_score = pd.DataFrame(index=avgWinScore.index, data=avgWinScore)
df_win_score.reset_index(inplace=True)
df_win_score = df_win_score.rename(columns={'WTeamID': 'TeamID', 'WScore': 'AvgWinScore'})

train_df = pd.merge(train_df, df_win_score, how='inner', on=['TeamID', 'Season'])

# Find average loss score
avgLossScore = reg_season_df.groupby(['Season', 'LTeamID'])['LScore'].mean()
df_loss_score = pd.DataFrame(index=avgLossScore.index, data=avgLossScore)
df_loss_score.reset_index(inplace=True)
df_loss_score = df_loss_score.rename(columns={'LTeamID': 'TeamID', 'LScore': 'AvgLossScore'})

train_df = pd.merge(train_df, df_loss_score, how='inner', on=['TeamID', 'Season'])

# Find Average Win Margin
avgWinMargin = reg_season_df.groupby(['Season', 'WTeamID'])['WScore'].mean() - reg_season_df.groupby(['Season', 'WTeamID'])['LScore'].mean()
df_win_margin = pd.DataFrame(index=avgWinMargin.index, data=avgWinMargin)
df_win_margin.reset_index(inplace=True)
df_win_margin.columns = ['Season', 'TeamID', 'AvgWinMargin']

train_df = pd.merge(train_df, df_win_margin, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Margin
avgLossMargin = reg_season_df.groupby(['Season', 'LTeamID'])['WScore'].mean() - reg_season_df.groupby(['Season', 'LTeamID'])['LScore'].mean()
df_loss_margin = pd.DataFrame(index=avgLossMargin.index, data=avgLossMargin)
df_loss_margin.reset_index(inplace=True)
df_loss_margin.columns = ['Season', 'TeamID', 'AvgLossMargin']

train_df = pd.merge(train_df, df_loss_margin, how='inner', on=['TeamID', 'Season'])

train_df

,Season,TeamID,NumWins,NumLosses,WinPercentage,AvgWinScore,AvgLossScore,AvgWinMargin,AvgLossMargin
0,1985,1102,5,19,0.208333,71.000000,61.000000,10.000000,9.947368
1,1985,1103,9,14,0.391304,70.222222,55.142857,7.555556,9.857143
2,1985,1104,21,9,0.700000,72.095238,60.111111,13.190476,4.777778
3,1985,1106,10,14,0.416667,75.100000,69.142857,9.500000,13.285714
4,1985,1108,19,6,0.760000,85.842105,74.000000,13.842105,10.666667
...,...,...,...,...,...,...,...,...,...
11219,2019,1462,18,15,0.545455,76.944444,65.800000,10.555556,9.866667
11220,2019,1463,21,7,0.750000,83.761905,72.285714,13.666667,12.285714
11221,2019,1464,10,20,0.333333,78.100000,71.200000,8.500000,12.850000
11222,2019,1465,12,14,0.461538,80.083333,71.500000,13.916667,11.428571


## Calculate Strength of Schedule

In [4]:
# Strength of Schedule = (2 * Opponents' SOS + Opoonents' Opponents' SOS) / 3
# SOS should help normalize win % based on how good a team's opponents are

team_sos = reg_season_df

# First find all Team IDs for opponents
team_sos = reg_season_df.groupby(['Season', 'WTeamID'])['LTeamID'].apply(list).reset_index(name='opponents')
team_sos = team_sos.rename(columns={'WTeamID': 'TeamID'})
temp = reg_season_df.groupby(['Season', 'LTeamID'])['WTeamID'].apply(list).reset_index(name='opponents2')
temp = temp.rename(columns={'LTeamID': 'TeamID'})
team_sos = pd.merge(team_sos, temp, how='inner', on=['TeamID', 'Season'])
team_sos['Opponents'] = team_sos['opponents'] + team_sos['opponents2']
team_sos = team_sos.drop(['opponents', 'opponents2'],  axis=1)

# Then find all Team IDs for opponents' opponents
# Find win percentage for all opponents' opponents

# Find average win percentage for all opponents
train_df = pd.merge(train_df, team_sos, how='inner', on=['Season', 'TeamID'])
winPercent = []
for i, row in train_df.iterrows():
    temp = train_df.loc[(train_df['Season'] == row['Season']) & (train_df['TeamID'].isin(row['Opponents']))]
    winPercent.append(temp['WinPercentage'].mean())
train_df['SOS'] = pd.Series(winPercent)
train_df = train_df.drop('Opponents',  axis=1)

# SOS = (2 * Opponents' SOS + Opoonents' Opponents' SOS) / 3

# Find adjusted win percentage based on sos
train_df['AdjustedWinPercentage'] = train_df['WinPercentage'] * train_df['SOS']
train_df

,Season,TeamID,NumWins,NumLosses,WinPercentage,AvgWinScore,AvgLossScore,AvgWinMargin,AvgLossMargin,SOS,AdjustedWinPercentage
0,1985,1102,5,19,0.208333,71.000000,61.000000,10.000000,9.947368,0.471618,0.098254
1,1985,1103,9,14,0.391304,70.222222,55.142857,7.555556,9.857143,0.462170,0.180849
2,1985,1104,21,9,0.700000,72.095238,60.111111,13.190476,4.777778,0.508611,0.356028
3,1985,1106,10,14,0.416667,75.100000,69.142857,9.500000,13.285714,0.496854,0.207023
4,1985,1108,19,6,0.760000,85.842105,74.000000,13.842105,10.666667,0.457361,0.347595
...,...,...,...,...,...,...,...,...,...,...,...
11219,2019,1462,18,15,0.545455,76.944444,65.800000,10.555556,9.866667,0.541957,0.295613
11220,2019,1463,21,7,0.750000,83.761905,72.285714,13.666667,12.285714,0.484091,0.363068
11221,2019,1464,10,20,0.333333,78.100000,71.200000,8.500000,12.850000,0.441693,0.147231
11222,2019,1465,12,14,0.461538,80.083333,71.500000,13.916667,11.428571,0.475953,0.219670


## Randomize Teams and Add Features

In [5]:
# Need to "randomize" the winning and losing teams for training
random_season_df = reg_season_df
sample = random_season_df.sample(frac=0.5)

temp = sample['WTeamID'].copy(deep=True)
sample['WTeamID'] = sample['LTeamID'].copy(deep=True)
sample['LTeamID'] = temp.copy(deep=True)

temp = sample['WScore'].copy(deep=True)
sample['WScore'] = sample['LScore'].copy(deep=True)
sample['LScore'] = temp.copy(deep=True)

random_season_df.iloc[sample.index.tolist(), :] = sample

# Create a result column "Spread" that is positive if W team wins and negative if L team wins.
# This will be used to train our model.
random_season_df['Spread'] = random_season_df['WScore'] - random_season_df['LScore']

# Add training data features for strong team
train_df = train_df.rename(columns={'TeamID': 'WTeamID'})
current_df = pd.merge(random_season_df, train_df, how='left', on=['Season', 'WTeamID'])
current_df = current_df.rename(columns=
                               {
                                   'NumWins': 'WNumWins',
                                   'NumLosses': 'WNumLosses',
                                   'WinPercentage': 'WWinPercentage',
                                   'AvgWinScore': 'WAvgWinScore',
                                   'AvgLossScore': 'WAvgLossScore',
                                   'AvgWinMargin': 'WAvgWinMargin',
                                   'AvgLossMargin': 'WAvgLossMargin',
                                   'SOS': 'WSOS',
                                   'AdjustedWinPercentage': 'WAdjustedWinPercentage'
                               })

# Add training data features for week team
train_df = train_df.rename(columns={'WTeamID': 'LTeamID'})
current_df = pd.merge(current_df, train_df, how='inner', on=['Season', 'LTeamID'])
current_df = current_df.rename(columns=
                               {
                                   'NumWins': 'LNumWins',
                                   'NumLosses': 'LNumLosses',
                                   'WinPercentage': 'LWinPercentage',
                                   'AvgWinScore': 'LAvgWinScore',
                                   'AvgLossScore': 'LAvgLossScore',
                                   'AvgWinMargin': 'LAvgWinMargin',
                                   'AvgLossMargin': 'LAvgLossMargin',
                                   'SOS': 'LSOS',
                                   'AdjustedWinPercentage': 'LAdjustedWinPercentage'
                               })

# Reset train_df
train_df = train_df.rename(columns={'LTeamID': 'TeamID'})

# Subtract relative columns (WNumWins - LNumWins, WAvgWinMargin - LAvgWinMargin, etc.)
current_df['NumWinsDifference'] = current_df['WNumWins'] - current_df['LNumWins']
current_df['NumLossesDifference'] = current_df['WNumLosses'] - current_df['LNumLosses']
current_df['AvgWinScoreDifference'] = current_df['WAvgWinScore'] - current_df['LAvgWinScore']
current_df['AvgLossScoreDifference'] = current_df['WAvgLossScore'] - current_df['LAvgLossScore']
current_df['AvgWinMarginDifference'] = current_df['WAvgWinMargin'] - current_df['LAvgWinMargin']
current_df['AvgLossMarginDifference'] = current_df['WAvgLossMargin'] - current_df['LAvgLossMargin']
current_df['WinPercentDifference'] = current_df['WWinPercentage'] - current_df['LWinPercentage']

# Drop unneeded columns
current_df = current_df.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
cols = current_df.columns.tolist()
current_df = current_df[cols[0:3] + cols[4:] + [cols[3]]]
current_df = current_df.drop(['Season', 'WTeamID', 'LTeamID'], axis=1)
#current_df = current_df.drop(['WNumWins', 'WNumLosses', 'LNumWins', 'LNumLosses'], axis=1)
current_df

,WNumWins,WNumLosses,WWinPercentage,WAvgWinScore,WAvgLossScore,WAvgWinMargin,WAvgLossMargin,WSOS,WAdjustedWinPercentage,LNumWins,...,LSOS,LAdjustedWinPercentage,NumWinsDifference,NumLossesDifference,AvgWinScoreDifference,AvgLossScoreDifference,AvgWinMarginDifference,AvgLossMarginDifference,WinPercentDifference,Spread
0,25.0,5.0,0.833333,92.800000,75.000000,17.960000,10.000000,0.549434,0.457861,23,...,0.555332,0.412021,2.0,-3.0,19.582609,21.125000,0.612174,2.250000,0.091398,-17
1,10.0,18.0,0.357143,87.100000,67.111111,14.900000,11.388889,0.505674,0.180598,23,...,0.555332,0.412021,-13.0,10.0,13.882609,13.236111,-2.447826,3.638889,-0.384793,-20
2,22.0,8.0,0.733333,67.590909,61.750000,8.045455,5.875000,0.535493,0.392695,23,...,0.555332,0.412021,-1.0,0.0,-5.626482,7.875000,-9.302372,-1.875000,-0.008602,7
3,25.0,5.0,0.833333,92.800000,75.000000,17.960000,10.000000,0.549434,0.457861,23,...,0.555332,0.412021,2.0,-3.0,19.582609,21.125000,0.612174,2.250000,0.091398,-3
4,12.0,13.0,0.480000,64.083333,55.307692,8.250000,11.076923,0.472093,0.226604,23,...,0.555332,0.412021,-11.0,5.0,-9.134058,1.432692,-9.097826,3.326923,-0.261935,-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161334,14.0,18.0,0.437500,75.285714,66.611111,12.571429,12.222222,0.596354,0.260905,18,...,0.574186,0.313192,-4.0,3.0,-1.936508,1.344444,3.293651,2.155556,-0.107955,1
161335,25.0,9.0,0.735294,81.800000,70.777778,17.120000,8.666667,0.567917,0.417586,18,...,0.574186,0.313192,7.0,-6.0,4.577778,5.511111,7.842222,-1.400000,0.189840,21
161336,9.0,23.0,0.281250,88.555556,60.130435,20.000000,13.695652,0.550070,0.154707,18,...,0.574186,0.313192,-9.0,8.0,11.333333,-5.136232,10.722222,3.628986,-0.264205,-10
161337,9.0,23.0,0.281250,88.555556,60.130435,20.000000,13.695652,0.550070,0.154707,18,...,0.574186,0.313192,-9.0,8.0,11.333333,-5.136232,10.722222,3.628986,-0.264205,-7


## Train Regression Model

In [6]:
# Train on regular season results using XGBoost regression model
X = current_df.iloc[:, :-1]
y = current_df.iloc[:, -1]

train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.2, random_state = 42) 

d_train = xgb.DMatrix(train_X, train_y) # format data properly
params = {
    'booster': 'gblinear',
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'eta': '.01',
    'max_depth': '2',
    'min_child_weight': '10'
}
watchlist= [(d_train, "train")]
num_boost_round = 500
model = xgb.train(params=params, dtrain=d_train, num_boost_round=num_boost_round, evals=watchlist, early_stopping_rounds=10, verbose_eval=20)
print("Finished Training")

model.save_model('./models/spreads.model')

[22:45:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mae:11.86131
[20]	train-mae:9.60383
[40]	train-mae:9.22404
[60]	train-mae:9.16429
[80]	train-mae:9.15078
[100]	train-mae:9.14431
[120]	train-mae:9.13944
[140]	train-mae:9.13547
[160]	train-mae:9.13216
[180]	train-mae:9.12938
[200]	train-mae:9.12699
[220]	train-mae:9.12480
[240]	train-mae:9.12279
[260]	train-mae:9.12085
[280]	train-mae:9.11898
[300]	train-mae:9.11712
[320]	train-mae:9.11531
[340]	train-mae:9.11347
[360]	train-mae:9.11163
[380]	train-mae:9.10976
[400]	train-mae:9.10788
[420]	train-mae:9.10598
[440]	train-mae:9.10406
[460]	train-mae:9.10214
[480]	train-mae:9.10018


## Parameter Tuning

In [7]:
# gridsearch_params = [
#     (max_depth, min_child_weight)
#     for max_depth in range(2, 4)
#     for min_child_weight in range(6, 11)
# ]

# # Define initial best params and MAE
# min_mae = float("Inf")
# best_params = None
# for max_depth, min_child_weight in gridsearch_params:
#     print("CV with max_depth={}, min_child_weight={}".format(
#                              max_depth,
#                              min_child_weight))
#     # Update our parameters
#     params['max_depth'] = max_depth
#     params['min_child_weight'] = min_child_weight
#     # Run CV
#     cv_results = xgb.cv(
#         params,
#         d_train,
#         num_boost_round=num_boost_round,
#         seed=42,
#         nfold=5,
#         metrics={'mae'},
#         early_stopping_rounds=10
#     )
#     # Update best MAE
#     mean_mae = cv_results['test-mae-mean'].min()
#     boost_rounds = cv_results['test-mae-mean'].argmin()
#     print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
#     if mean_mae < min_mae:
#         min_mae = mean_mae
#         best_params = (max_depth,min_child_weight)
# print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

## Test Regression Model

In [8]:
# Test model
d_test = xgb.DMatrix(test_X)
pred = model.predict(d_test)
print("Predictions: ", pred)

rmse = mean_squared_error(test_y, pred, squared=False)
print("Root Mean Squared Error : %f \n" %(rmse))

mae = mean_absolute_error(test_y, pred)
print("Mean Absolute Error : %f \n" %(mae))

r2 = r2_score(test_y, pred)
print("R-Squared : %f \n" %(r2))

Predictions:  [22.914497   1.1035591  1.0956132 ... -5.0223207  5.814123  -1.8711774]
Root Mean Squared Error : 11.550301 

Mean Absolute Error : 9.134411 

R-Squared : 0.428889 



Using regular season compact results, we achieve a rmse of 11.54, mae of 9.09, and r2 of 0.44

## Read Detailed Data

In [120]:
# Load regular season detailed results
reg_season_detailed_df = pd.read_csv('./data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
reg_season_detailed_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

## Create Features

In [121]:
## Create Features

# Find number of wins for each team in each season
numWins = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['DayNum'].nunique()
df_win = pd.DataFrame(index=numWins.index, data=numWins)
df_win.reset_index(inplace=True)
df_win = df_win.rename(columns={'WTeamID': 'TeamID', 'DayNum': 'NumWins'})

# Find number of losses for each team
numLosses = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['DayNum'].nunique()
df_loss = pd.DataFrame(index=numLosses.index, data=numLosses)
df_loss.reset_index(inplace=True)
df_loss = df_loss.rename(columns={'LTeamID': 'TeamID', 'DayNum': 'NumLosses'})

train_df = pd.merge(df_win, df_loss, how='inner', on=['TeamID', 'Season'])

# Find average win score
avgWinScore = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WScore'].mean()
df_win_score = pd.DataFrame(index=avgWinScore.index, data=avgWinScore)
df_win_score.reset_index(inplace=True)
df_win_score = df_win_score.rename(columns={'WTeamID': 'TeamID', 'WScore': 'AvgWinScore'})

train_df = pd.merge(train_df, df_win_score, how='inner', on=['TeamID', 'Season'])

# Find average loss score
avgLossScore = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LScore'].mean()
df_loss_score = pd.DataFrame(index=avgLossScore.index, data=avgLossScore)
df_loss_score.reset_index(inplace=True)
df_loss_score = df_loss_score.rename(columns={'LTeamID': 'TeamID', 'LScore': 'AvgLossScore'})

train_df = pd.merge(train_df, df_loss_score, how='inner', on=['TeamID', 'Season'])

# Find Average Win Margin
avgWinMargin = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WScore'].mean() - reg_season_detailed_df.groupby(['Season', 'WTeamID'])['LScore'].mean()
df_win_margin = pd.DataFrame(index=avgWinMargin.index, data=avgWinMargin)
df_win_margin.reset_index(inplace=True)
df_win_margin.columns = ['Season', 'TeamID', 'AvgWinMargin']

train_df = pd.merge(train_df, df_win_margin, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Margin
avgLossMargin = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['WScore'].mean() - reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LScore'].mean()
df_loss_margin = pd.DataFrame(index=avgLossMargin.index, data=avgLossMargin)
df_loss_margin.reset_index(inplace=True)
df_loss_margin.columns = ['Season', 'TeamID', 'AvgLossMargin']

train_df = pd.merge(train_df, df_loss_margin, how='inner', on=['TeamID', 'Season'])

# Find Average Win Field Goals Made
avgWFGM = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGM'].mean()
df_wfgm = pd.DataFrame(index=avgWFGM.index, data=avgWFGM)
df_wfgm.reset_index(inplace=True)
df_wfgm = df_wfgm.rename(columns={'WTeamID': 'TeamID', 'WFGM': 'AvgWinFGM'})

train_df = pd.merge(train_df, df_wfgm, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Field Goals Made
avgLFGM = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGM'].mean()
df_lfgm = pd.DataFrame(index=avgLFGM.index, data=avgLFGM)
df_lfgm.reset_index(inplace=True)
df_lfgm = df_lfgm.rename(columns={'LTeamID': 'TeamID', 'LFGM': 'AvgLossFGM'})

train_df = pd.merge(train_df, df_lfgm, how='inner', on=['TeamID', 'Season'])

# Find Average Win Field Goals Made/Attempted %
avgWFGMRatio = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGM'].mean() / reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGA'].mean()
df_wfgm_ratio = pd.DataFrame(index=avgWFGMRatio.index, data=avgWFGMRatio)
df_wfgm_ratio.reset_index(inplace=True)
df_wfgm_ratio.columns = ['Season', 'TeamID', 'AvgWinFGMRatio']

train_df = pd.merge(train_df, df_wfgm_ratio, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Field Goals Made/Attempted %
avgLFGMRatio = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGM'].mean() / reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGA'].mean()
df_lfgm_ratio = pd.DataFrame(index=avgLFGMRatio.index, data=avgLFGMRatio)
df_lfgm_ratio.reset_index(inplace=True)
df_lfgm_ratio.columns = ['Season', 'TeamID', 'AvgLossFGMRatio']

train_df = pd.merge(train_df, df_lfgm_ratio, how='inner', on=['TeamID', 'Season'])

# Find Average Win 3 Pointers Made
avgWFGM3 = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGM3'].mean()
df_wfgm3 = pd.DataFrame(index=avgWFGM3.index, data=avgWFGM3)
df_wfgm3.reset_index(inplace=True)
df_wfgm3 = df_wfgm3.rename(columns={'WTeamID': 'TeamID', 'WFGM3': 'AvgWinFGM3'})

train_df = pd.merge(train_df, df_wfgm3, how='inner', on=['TeamID', 'Season'])

# Find Average Loss 3 Pointers Made
avgLFGM3 = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGM3'].mean()
df_lfgm3 = pd.DataFrame(index=avgLFGM3.index, data=avgLFGM3)
df_lfgm3.reset_index(inplace=True)
df_lfgm3 = df_lfgm3.rename(columns={'LTeamID': 'TeamID', 'LFGM3': 'AvgLossFGM3'})

train_df = pd.merge(train_df, df_lfgm3, how='inner', on=['TeamID', 'Season'])

# Find Average Win 3 Pointers Made/Attempted %
avgWFGM3Ratio = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGM3'].mean() / reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFGA3'].mean()
df_wfgm3_ratio = pd.DataFrame(index=avgWFGM3Ratio.index, data=avgWFGM3Ratio)
df_wfgm3_ratio.reset_index(inplace=True)
df_wfgm3_ratio.columns = ['Season', 'TeamID', 'AvgWinFGM3Ratio']

train_df = pd.merge(train_df, df_wfgm3_ratio, how='inner', on=['TeamID', 'Season'])

# Find Average Loss 3 Pointers Made/Attempted %
avgLFGM3Ratio = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGM3'].mean() / reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFGA3'].mean()
df_lfgm3_ratio = pd.DataFrame(index=avgLFGM3Ratio.index, data=avgLFGM3Ratio)
df_lfgm3_ratio.reset_index(inplace=True)
df_lfgm3_ratio.columns = ['Season', 'TeamID', 'AvgLossFGM3Ratio']

train_df = pd.merge(train_df, df_lfgm3_ratio, how='inner', on=['TeamID', 'Season'])

# Find Average Win Offensive Rebounds
avgWOR = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WOR'].mean()
df_wor = pd.DataFrame(index=avgWOR.index, data=avgWOR)
df_wor.reset_index(inplace=True)
df_wor = df_wor.rename(columns={'WTeamID': 'TeamID', 'WOR': 'AvgWinOffReb'})

train_df = pd.merge(train_df, df_wor, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Offensive Rebounds
avgLOR = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LOR'].mean()
df_lor = pd.DataFrame(index=avgLOR.index, data=avgLOR)
df_lor.reset_index(inplace=True)
df_lor = df_lor.rename(columns={'LTeamID': 'TeamID', 'LOR': 'AvgLossOffReb'})

train_df = pd.merge(train_df, df_lor, how='inner', on=['TeamID', 'Season'])

# Find Average Win Defensive Rebounds
avgWDR = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WDR'].mean()
df_wdr = pd.DataFrame(index=avgWDR.index, data=avgWDR)
df_wdr.reset_index(inplace=True)
df_wdr = df_wdr.rename(columns={'WTeamID': 'TeamID', 'WDR': 'AvgWinDefReb'})

train_df = pd.merge(train_df, df_wdr, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Defensive Rebounds
avgLDR = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LDR'].mean()
df_ldr = pd.DataFrame(index=avgLDR.index, data=avgLDR)
df_ldr.reset_index(inplace=True)
df_ldr = df_ldr.rename(columns={'LTeamID': 'TeamID', 'LDR': 'AvgLossDefReb'})

train_df = pd.merge(train_df, df_ldr, how='inner', on=['TeamID', 'Season'])

# Find Average Win Free Throws Made
avgWFTM = reg_season_detailed_df.groupby(['Season', 'WTeamID'])['WFTM'].mean()
df_wftm = pd.DataFrame(index=avgWFTM.index, data=avgWFTM)
df_wftm.reset_index(inplace=True)
df_wftm = df_wftm.rename(columns={'WTeamID': 'TeamID', 'WFTM': 'AvgWinFTM'})

train_df = pd.merge(train_df, df_wftm, how='inner', on=['TeamID', 'Season'])

# Find Average Loss Free Throws Made
avgLFTM = reg_season_detailed_df.groupby(['Season', 'LTeamID'])['LFTM'].mean()
df_lftm = pd.DataFrame(index=avgLFTM.index, data=avgLFTM)
df_lftm.reset_index(inplace=True)
df_lftm = df_lftm.rename(columns={'LTeamID': 'TeamID', 'LFTM': 'AvgLossFTM'})

train_df = pd.merge(train_df, df_lftm, how='inner', on=['TeamID', 'Season'])

train_df

,Season,TeamID,NumWins,NumLosses,AvgWinScore,AvgLossScore,AvgWinMargin,AvgLossMargin,AvgWinFGM,AvgLossFGM,...,AvgWinFGM3,AvgLossFGM3,AvgWinFGM3Ratio,AvgLossFGM3Ratio,AvgWinOffReb,AvgLossOffReb,AvgWinDefReb,AvgLossDefReb,AvgWinFTM,AvgLossFTM
0,2003,1102,12,16,68.750000,48.625000,15.583333,11.250000,22.583333,16.562500,...,10.000000,6.187500,0.463320,0.305556,3.833333,4.437500,19.333333,14.937500,13.583333,9.312500
1,2003,1103,13,14,87.769231,70.428571,9.384615,7.500000,30.000000,24.500000,...,5.461538,5.428571,0.379679,0.307692,9.384615,10.142857,21.461538,18.500000,22.307692,16.000000
2,2003,1104,17,11,74.705882,60.909091,13.176471,9.454545,25.823529,21.272727,...,7.058824,5.272727,0.338983,0.287129,13.529412,13.636364,26.411765,20.090909,16.000000,13.090909
3,2003,1105,7,19,79.428571,68.947368,13.000000,11.473684,25.571429,23.947368,...,9.142857,7.000000,0.407643,0.347258,14.571429,13.105263,25.857143,22.105263,19.142857,14.052632
4,2003,1106,13,15,68.307692,59.533333,10.384615,9.266667,24.769231,22.266667,...,5.846154,6.333333,0.367150,0.331010,12.769231,11.866667,28.000000,20.266667,12.923077,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5822,2019,1462,18,15,76.944444,65.800000,10.555556,9.866667,27.666667,24.266667,...,7.555556,6.333333,0.359788,0.300633,10.277778,10.600000,27.944444,23.133333,14.055556,10.933333
5823,2019,1463,21,7,83.761905,72.285714,13.666667,12.285714,31.000000,26.285714,...,8.000000,7.142857,0.400000,0.306748,9.000000,7.714286,31.095238,26.000000,13.761905,12.571429
5824,2019,1464,10,20,78.100000,71.200000,8.500000,12.850000,28.000000,26.250000,...,10.100000,9.300000,0.365942,0.329787,12.300000,13.300000,25.200000,23.400000,12.000000,9.400000
5825,2019,1465,12,14,80.083333,71.500000,13.916667,11.428571,27.416667,24.857143,...,9.583333,8.142857,0.393836,0.313187,9.416667,10.642857,27.166667,25.500000,15.666667,13.642857


In [122]:
# Need to "randomize" the winning and losing teams for training
random_season_detailed_df = reg_season_detailed_df
sample = random_season_detailed_df.sample(frac=0.5)

temp = sample['WTeamID'].copy(deep=True)
sample['WTeamID'] = sample['LTeamID'].copy(deep=True)
sample['LTeamID'] = temp.copy(deep=True)

temp = sample['WScore'].copy(deep=True)
sample['WScore'] = sample['LScore'].copy(deep=True)
sample['LScore'] = temp.copy(deep=True)

random_season_detailed_df.iloc[sample.index.tolist(), :] = sample

random_season_detailed_df['Spread'] = random_season_df['WScore'] - random_season_df['LScore']

# Add training data features for strong team
train_df = train_df.rename(columns={'TeamID': 'WTeamID'})
current_detailed_df = pd.merge(random_season_detailed_df, train_df, how='left', on=['Season', 'WTeamID'])
current_detailed_df = current_detailed_df.rename(columns=
                               {
                                   'NumWins': 'WNumWins',
                                   'NumLosses': 'WNumLosses',
                                   'AvgWinScore': 'WAvgWinScore',
                                   'AvgLossScore': 'WAvgLossScore',
                                   'AvgWinMargin': 'WAvgWinMargin',
                                   'AvgLossMargin': 'WAvgLossMargin',
                                   'AvgWinFGM': 'WAvgWinFGM',
                                   'AvgLossFGM': 'WAvgLossFGM',
                                   'AvgWinFGMRatio': 'WAvgWinFGMRatio',
                                   'AvgLossFGMRatio': 'WAvgLossFGMRatio',
                                   'AvgWinFGM3': 'WAvgWinFGM3',
                                   'AvgLossFGM3': 'WAvgLossFGM3',
                                   'AvgWinFGM3Ratio': 'WAvgWinFGM3Ratio',
                                   'AvgLossFGM3Ratio': 'WAvgLossFGM3Ratio',
                                   'AvgWinOffReb': 'WAvgWinOffReb',
                                   'AvgLossOffReb': 'WAvgLossOffReb',
                                   'AvgWinDefReb': 'WAvgWinDefReb',
                                   'AvgLossDefReb': 'WAvgLossDefReb',
                                   'AvgWinFTM': 'WAvgWinFTM',
                                   'AvgLossFTM': 'WAvgLossFTM'
                               })

# Add training data features for week team
train_df = train_df.rename(columns={'WTeamID': 'LTeamID'})
current_detailed_df = pd.merge(current_detailed_df, train_df, how='inner', on=['Season', 'LTeamID'])
current_detailed_df = current_detailed_df.rename(columns=
                               {
                                   'NumWins': 'LNumWins',
                                   'NumLosses': 'LNumLosses',
                                   'AvgWinScore': 'LAvgWinScore',
                                   'AvgLossScore': 'LAvgLossScore',
                                   'AvgWinMargin': 'LAvgWinMargin',
                                   'AvgLossMargin': 'LAvgLossMargin',
                                   'AvgWinFGM': 'LAvgWinFGM',
                                   'AvgLossFGM': 'LAvgLossFGM',
                                   'AvgWinFGMRatio': 'LAvgWinFGMRatio',
                                   'AvgLossFGMRatio': 'LAvgLossFGMRatio',
                                   'AvgWinFGM3': 'LAvgWinFGM3',
                                   'AvgLossFGM3': 'LAvgLossFGM3',
                                   'AvgWinFGM3Ratio': 'LAvgWinFGM3Ratio',
                                   'AvgLossFGM3Ratio': 'LAvgLossFGM3Ratio',
                                   'AvgWinOffReb': 'LAvgWinOffReb',
                                   'AvgLossOffReb': 'LAvgLossOffReb',
                                   'AvgWinDefReb': 'LAvgWinDefReb',
                                   'AvgLossDefReb': 'LAvgLossDefReb',
                                   'AvgWinFTM': 'LAvgWinFTM',
                                   'AvgLossFTM': 'LAvgLossFTM'
                               })

# Reset train_df
train_df = train_df.rename(columns={'LTeamID': 'TeamID'})

# Subtract relative columns (WNumWins - LNumWins, WAvgWinMargin - LAvgWinMargin, etc.)
current_detailed_df['NumWinsDifference'] = current_detailed_df['WNumWins'] - current_detailed_df['LNumWins']
current_detailed_df['NumLossesDifference'] = current_detailed_df['WNumLosses'] - current_detailed_df['LNumLosses']
current_detailed_df['AvgWinsScoreDifference'] = current_detailed_df['WAvgWinScore'] - current_detailed_df['LAvgWinScore']
current_detailed_df['AvgLossScoreDifference'] = current_detailed_df['WAvgLossScore'] - current_detailed_df['LAvgLossScore']
current_detailed_df['AvgWinMarginDifference'] = current_detailed_df['WAvgWinMargin'] - current_detailed_df['LAvgWinMargin']
current_detailed_df['AvgLossMarginDifference'] = current_detailed_df['WAvgLossMargin'] - current_detailed_df['LAvgLossMargin']
current_detailed_df['AvgWinOffRebDifference'] = current_detailed_df['WAvgWinOffReb'] - current_detailed_df['LAvgWinOffReb']
current_detailed_df['AvgLossOffRebDifference'] = current_detailed_df['WAvgLossOffReb'] - current_detailed_df['LAvgLossOffReb']
current_detailed_df['AvgWinFGMRatioDifference'] = current_detailed_df['WAvgWinFGMRatio'] - current_detailed_df['LAvgWinFGMRatio']
current_detailed_df['AvgWinDefRebDifference'] = current_detailed_df['WAvgWinDefReb'] - current_detailed_df['LAvgWinDefReb']
current_detailed_df['AvgLossDefRebDifference'] = current_detailed_df['WAvgLossDefReb'] - current_detailed_df['LAvgLossDefReb']
current_detailed_df['AvgWinFTMDifference'] = current_detailed_df['WAvgWinFTM'] - current_detailed_df['LAvgWinFTM']
current_detailed_df['AvgLossFTMDifference'] = current_detailed_df['WAvgLossFTM'] - current_detailed_df['LAvgLossFTM']

# Drop unneeded columns
current_detailed_df = current_detailed_df.drop(
    [
        'DayNum',
        'WScore',
        'LScore',
        'WLoc',
        'NumOT',
        'WFGM',
        'WFGA',
        'WFGM3',
        'WFGA3',
        'WFTM',
        'WFTA',
        'WOR',
        'WDR',
        'WAst',
        'WTO',
        'WStl',
        'WBlk',
        'WPF',
        'LFGM',
        'LFGA',
        'LFGM3',
        'LFGA3',
        'LFTM',
        'LFTA',
        'LOR',
        'LDR',
        'LAst',
        'LTO',
        'LStl',
        'LBlk',
        'LPF'
    ], axis=1)
cols = current_detailed_df.columns.tolist()
current_detailed_df = current_detailed_df[cols[0:3] + cols[4:] + [cols[3]]]
current_detailed_df = current_detailed_df.drop(['Season', 'WTeamID', 'LTeamID'], axis=1)
current_detailed_df = current_detailed_df.drop(
    [
        'WNumWins',
        'WNumLosses',
        'LNumWins',
        'LNumLosses',
        'WAvgWinScore',
        'LAvgWinScore',
        'WAvgLossScore',
        'LAvgLossScore',
        'WAvgWinMargin',
        'WAvgLossMargin',
        'WAvgWinFGM',
        'WAvgLossFGM',
        'WAvgWinFGMRatio',
        'WAvgLossFGMRatio',
        'WAvgWinFGM3',
        'WAvgLossFGM3',
        'WAvgWinFGM3Ratio',
        'WAvgLossFGM3Ratio',
        'LAvgWinMargin',
        'LAvgLossMargin',
        'LAvgWinFGM',
        'LAvgLossFGM',
        'LAvgWinFGMRatio',
        'LAvgLossFGMRatio',
        'LAvgWinFGM3',
        'LAvgLossFGM3',
        'LAvgWinFGM3Ratio',
        'LAvgLossFGM3Ratio'
    ], axis=1)

current_detailed_df

,WAvgWinOffReb,WAvgLossOffReb,WAvgWinDefReb,WAvgLossDefReb,WAvgWinFTM,WAvgLossFTM,LAvgWinOffReb,LAvgLossOffReb,LAvgWinDefReb,LAvgLossDefReb,...,AvgWinMarginDifference,AvgLossMarginDifference,AvgWinOffRebDifference,AvgLossOffRebDifference,AvgWinFGMRatioDifference,AvgWinDefRebDifference,AvgLossDefRebDifference,AvgWinFTMDifference,AvgLossFTMDifference,Spread
0,11.833333,13.333333,25.583333,22.500000,13.666667,11.166667,13.529412,13.636364,26.411765,20.090909,...,2.365196,-2.287879,-1.696078,-0.303030,0.024907,-0.828431,2.409091,-2.333333,-1.924242,17
1,12.769231,11.866667,28.000000,20.266667,12.923077,8.666667,13.529412,13.636364,26.411765,20.090909,...,-2.791855,-0.187879,-0.760181,-1.769697,0.017460,1.588235,0.175758,-3.076923,-4.424242,4
2,9.785714,11.142857,27.785714,21.571429,13.785714,11.785714,13.529412,13.636364,26.411765,20.090909,...,-1.819328,1.759740,-3.743697,-2.493506,0.057460,1.373950,1.480519,-2.214286,-1.305195,-22
3,10.705882,12.642857,25.352941,20.142857,17.058824,16.428571,13.529412,13.636364,26.411765,20.090909,...,-3.588235,2.259740,-2.823529,-0.993506,0.016364,-1.058824,0.051948,1.058824,3.337662,5
4,7.545455,9.437500,23.181818,21.375000,20.636364,15.812500,13.529412,13.636364,26.411765,20.090909,...,-6.449198,2.170455,-5.983957,-4.198864,0.040890,-3.229947,1.284091,4.636364,2.721591,-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87393,9.846154,10.157895,28.000000,24.105263,15.153846,14.000000,11.562500,12.200000,26.687500,23.266667,...,-4.187500,2.056140,-1.716346,-2.042105,-0.006586,1.312500,0.838596,-1.158654,0.400000,-45
87394,8.823529,8.800000,27.294118,23.666667,16.294118,15.333333,11.562500,12.200000,26.687500,23.266667,...,-1.363971,-0.200000,-2.738971,-3.400000,0.010232,0.606618,0.400000,-0.018382,1.733333,4
87395,8.466667,10.235294,24.733333,20.117647,13.666667,11.352941,11.562500,12.200000,26.687500,23.266667,...,-6.587500,2.737255,-3.095833,-1.964706,-0.005786,-1.954167,-3.149020,-2.645833,-2.247059,7
87396,9.750000,10.714286,27.562500,23.571429,16.937500,13.285714,11.562500,12.200000,26.687500,23.266667,...,-1.375000,-1.090476,-1.812500,-1.485714,-0.000484,0.875000,0.304762,0.625000,-0.314286,-23


In [123]:
# Train on regular season results using XGBoost regression model
X = current_detailed_df.iloc[:, :-1]
y = current_detailed_df.iloc[:, -1]

train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.2, random_state = 42) 

d_train = xgb.DMatrix(train_X, train_y) # format data properly
params = {
    'booster': 'gblinear',
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'eta': '.01',
    'max_depth': '16'
}
watchlist= [(d_train, "train")]
num_boost_round = 500
model = xgb.train(params=params, dtrain=d_train, num_boost_round=num_boost_round, evals=watchlist, early_stopping_rounds=10, verbose_eval=20)
print("Finished Training")

[14:29:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mae:12.21003
[10]	train-mae:12.21085
Finished Training


In [124]:
# Test model
d_test = xgb.DMatrix(test_X)
pred = model.predict(d_test)
print("Predictions: ", pred)

rmse = mean_squared_error(test_y, pred, squared=False)
print("Root Mean Squared Error : %f \n" %(rmse))

mae = mean_absolute_error(test_y, pred)
print("Mean Absolute Error : %f \n" %(mae))

r2 = r2_score(test_y, pred)
print("R-Squared : %f \n" %(r2))

Predictions:  [ 0.23230773  0.09109415 -0.01749951 ...  0.09413007  0.06351143
  0.11203244]
Root Mean Squared Error : 15.527128 

Mean Absolute Error : 12.198155 

R-Squared : -0.000138 

